<a href="https://colab.research.google.com/github/GuCuChiara/BERT-en-Spanish/blob/main/BETO_preguntas_y_respuestas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT en Español: preguntas y respuestas

# BETO (Spanish BERT) + Spanish SQuAD2.0 + distillation using 'bert-base-multilingual-cased' as teacher
This model was fine-tuned on the same dataset but using distillation during the process as mentioned above (and one more train epoch).

The teacher model for the distillation was bert-base-multilingual-cased. It is the same teacher used for distilbert-base-multilingual-cased AKA DistilmBERT (on average is twice as fast as mBERT-base).

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/',force_remount=True)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 182 kB 56.6 MB/s 
     |████████████████████████████████| 7.6 MB 35.6 MB/s 


In [ ]:
# Importar
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
the_model = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
tokenizer = AutoTokenizer.from_pretrained(the_model, do_lower_case=False)
model = AutoModelForQuestionAnswering.from_pretrained(the_model)

Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
print(model)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
# Ejemplo tokenización
contexto = 'Argentina ganó la copa del mundo de fútbol en el año 2022'
pregunta = '¿quién ganó la copa del mundo en el año 2022?'

encode = tokenizer.encode_plus(pregunta, contexto, return_tensors='pt')
input_ids = encode['input_ids'].tolist()
print(input_ids)
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
for id, token in zip(input_ids[0], tokens):
  print('{:<12} {:>6}'.format(token, id))
  print('')

[[4, 1067, 2461, 6820, 1030, 9216, 1072, 1863, 1036, 1040, 1730, 22346, 30973, 1064, 5, 5151, 6820, 1030, 9216, 1072, 1863, 1008, 5921, 1036, 1040, 1730, 22346, 30973, 5]]
[CLS]             4

¿              1067

quién          2461

ganó           6820

la             1030

copa           9216

del            1072

mundo          1863

en             1036

el             1040

año            1730

202           22346

##2           30973

?              1064

[SEP]             5

Argentina      5151

ganó           6820

la             1030

copa           9216

del            1072

mundo          1863

de             1008

fútbol         5921

en             1036

el             1040

año            1730

202           22346

##2           30973

[SEP]             5



In [ ]:
# Ejemplo de inferencia (pregunta-respuesta)
#Pipeline: tipo de tarea, modelo elejido y cuál es el tokenizer
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
#le especificamos al pipeline  la pregunta y el contexto que escribimos arriba
salida = nlp({'question':pregunta, 'context':contexto})
#mostramos le inferencia
print(salida)

{'score': 0.9753745198249817, 'start': 0, 'end': 9, 'answer': 'Argentina'}


In [ ]:
from textwrap import wrap

def pregunta_respuesta(model, contexto, nlp):

  # Imprimir contexto
  print('Contexto:')
  print('-----------------')
  print('\n'.join(wrap(contexto)))

  # Loop preguntas-respuestas:
  continuar = True
  while continuar:
    print('\nPregunta:')
    print('-----------------')
    pregunta = str(input())

    continuar = pregunta!=''

    if continuar:
      salida = nlp({'question':pregunta, 'context':contexto})
      print('\nRespuesta:')
      print('-----------------')
      print(salida['answer'])


In [ ]:
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
Argentina ganó la copa del mundo de fútbol en el año 2022

Pregunta:
-----------------



In [ ]:
contexto = "Lionel Andrés Messi Cuccittini (Rosario, 24 de junio de 1987), conocido como Leo Messi, es un futbolista argentino que juega como delantero o centrocampista. Jugador histórico del Fútbol Club Barcelona, al que estuvo ligado veinte años, desde 2021 integra el plantel del Paris Saint-Germain de la Ligue 1 de Francia. Es también internacional con la selección de Argentina, equipo del que es capitán. Considerado con frecuencia el mejor jugador del mundo y uno de los mejores de todos los tiempos,9​ es el único futbolista en la historia que ha ganado, entre otras distinciones, siete veces el Balón de Oro, seis premios de la FIFA al mejor jugador del mundo, seis Botas de Oro y dos Balones de Oro de la Copa Mundial de Fútbol. En 2020, se convirtió en el primer futbolista y el primer argentino en recibir un premio Laureus, además de ser incluido en el Dream Team del Balón de Oro.Con el Barcelona ha ganado 35 títulos, entre ellos, diez de La Liga, cuatro de la Liga de Campeones de la UEFA y siete de la Copa del Rey. Goleador prolífico, ostenta, entre otros, los récords por más goles en una temporada,10​ en un mismo club y en un año calendario: en 2012, superó la marca de Gerd Müller e ingresó en el Guinness World Records. Es, además, el máximo goleador histórico del Barcelona y de la selección argentina, de La Liga, la Supercopa de España, la Supercopa de Europa y el jugador no europeo con más goles en la Liga de Campeones de la UEFA. Nacido y criado en la ciudad de Rosario, a los 13 años se radicó en España, donde el Barcelona accedió a pagar el tratamiento de la enfermedad hormonal que le habían diagnosticado de niño. Después de una rápida progresión por la Academia juvenil del Barcelona, hizo su debut oficial con el primer equipo en octubre de 2004, a los diecisiete años. A pesar de ser propenso a lesiones en los inicios de su carrera, ya en 2006 se estableció como jugador fundamental para el club. Su primera campaña ininterrumpida fue la temporada 2008-09, en la que el Barcelona alcanzó el primer triplete del fútbol español. Por su estilo de juego de pequeño driblador zurdo,11​ pronto se lo comparó con su compatriota Diego Maradona quien, en 2007, lo declaró su «sucesor». En 2009, a los veintidós años, ganó su primer Balón de Oro y el premio al Jugador Mundial de la FIFA del año. Siguieron tres temporadas exitosas, en las que ganó cuatro Balones de Oro de forma consecutiva, hecho que no tenía precedentes. Hasta el momento, su mejor campaña personal es la temporada 2011-12, en la que estableció el récord de más goles en una temporada, tanto en La Liga como en otras competiciones europeas. Durante las dos siguientes temporadas, también sufrió lesiones y, en 2014, perdió el Balón de Oro frente a Cristiano Ronaldo, a quien se considera su rival. Recuperó su mejor forma durante la campaña 2014-15, en la que superó los registros de máximo goleador absoluto en La Liga y la Liga de Campeones y logró con el Barcelona un histórico segundo triplete, además de ganar su quinto Balón de Oro. Volvería a ganarlo por sexta y séptima vez en 2019 y 2021.Como internacional argentino, ha representado a su país en once torneos mayores. A nivel juvenil, en 2005 participó con la selección sub-20 en el Sudamericano de Colombia y ganó la Copa Mundial de Países Bajos, torneo en el que finalizó como mejor jugador y máximo goleador y, con la sub-23, recibió la medalla de oro en los Juegos Olímpicos de 2008. Después de debutar en la selección mayor en agosto de 2005, en el Mundial de Alemania 2006 se convirtió en el argentino más joven en jugar y en marcar en un mundial. Al año siguiente, en la Copa América, fue nombrado mejor jugador joven del torneo. Como capitán desde agosto de 2011, llegó con su equipo a las finales del Mundial de Brasil 2014, de la Copa América 2015 (certámenes en los que fue elegido mejor jugador del torneo) y de la Copa América Centenario. La selección argentina ganó la Copa América 2021 ante Brasil en el Maracaná y, en 2022, la Finalissima ante Italia en Wembley y el Mundial de Catar contra Francia en el estadio Lusail."
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
Lionel Andrés Messi Cuccittini (Rosario, 24 de junio de 1987),
conocido como Leo Messi, es un futbolista argentino que juega como
delantero o centrocampista. Jugador histórico del Fútbol Club
Barcelona, al que estuvo ligado veinte años, desde 2021 integra el
plantel del Paris Saint-Germain de la Ligue 1 de Francia. Es también
internacional con la selección de Argentina, equipo del que es
capitán. Considerado con frecuencia el mejor jugador del mundo y uno
de los mejores de todos los tiempos,9​ es el único futbolista en la
historia que ha ganado, entre otras distinciones, siete veces el Balón
de Oro, seis premios de la FIFA al mejor jugador del mundo, seis Botas
de Oro y dos Balones de Oro de la Copa Mundial de Fútbol. En 2020, se
convirtió en el primer futbolista y el primer argentino en recibir un
premio Laureus, además de ser incluido en el Dream Team del Balón de
Oro.Con el Barcelona ha ganado 35 títulos, entre ellos, diez de La
Liga, cuatro de la Liga de

In [ ]:
contexto = "La diabetes mellitus (DM) es un conjunto de trastornos metabólicos, cuya característica común principal es la presencia de concentraciones elevadas de glucosa en la sangre de manera persistente o crónica, debido ya sea a un defecto en la producción de insulina, a una resistencia a la acción de ella para utilizar la glucosa, a un aumento en la producción de glucosa o a una combinación de estas causas. También se acompaña de anormalidades en el metabolismo de los lípidos, proteínas, sales minerales y electrolitos.La diabetes se asocia con la aparición de complicaciones en muchos sistemas orgánicos, siendo los más evidentes la pérdida de la visión (retinopatía diabética) que puede llegar a la ceguera, el compromiso de los riñones (nefropatía diabética) con deterioro funcional progresivo, requiriendo diálisis y trasplante, el compromiso de vasos sanguíneos que pueden significar la pérdida de extremidades inferiores (amputaciones) (véase vasculopatía diabética), el compromiso del corazón con enfermedad coronaria e infarto agudo de miocardio, el compromiso cerebral y la irrigación intestinal; sin embargo, las complicaciones más prevalentes afectan al sistema nervioso periférico y autónomo. Todo esto significa una carga muy pesada para el paciente que la padece y para todo el sistema de salud pública.Los síntomas principales de la diabetes mellitus son la emisión excesiva de orina (poliuria), el aumento anormal de la necesidad de comer (polifagia), el incremento de la sed (polidipsia) y la pérdida de peso sin razón aparente.14​ En ocasiones se toman como referencia estos tres síntomas (poliuria, polifagia y polidipsia o regla de las 3 P) para poder sospechar diabetes mellitus tipo 2 o insulinorresistente, ya que son los más comunes en la población. Además de estos síntomas, también está el cansancio, el dolor de cabeza, el mareo y vista encandilada.Una revisión de 2018 llega a la conclusión de que uno de los posibles factores de riesgo para desarrollar tanto la diabetes tipo 1 es el gluten y que la dieta sin gluten durante el embarazo reduce el riesgo de que el bebé desarrolle diabetes tipo 1.Actualmente existen dos clasificaciones principales. La primera, correspondiente a la Organización Mundial de la Salud, en la que reconocen tres formas de diabetes mellitus: tipo 1, tipo 2 y diabetes gestacional (ocurre durante el embarazo), cada una con diferentes causas y con distinta incidencia7​10​ y la segunda, propuesta por el comité de expertos de la Asociación Americana de Diabetes (ADA) en 1997. Según la ADA, los diferentes tipos de DM se clasifican en cuatro grupos, La diabetes mellitus tipo 1, en la que existe una destrucción total (mayor del 90 %) de las células β, lo que conlleva una deficiencia absoluta de insulina. Representa entre un 5 a un 10 % de las DM.La diabetes mellitus tipo 2, generada como consecuencia de un defecto progresivo en la secreción de insulina, de un aumento de la resistencia periférica a la misma y de un aumento de la producción de glucosa. Representa entre un 90 a un 95 % de las DM.La diabetes gestacional, que es diagnosticada durante el segundo o tercer trimestre del embarazo, donde muchas veces es transitoria y la propia cura es el parto.Diabetes producida por otras causas: Defectos genéticos de la función de las células β (por ejemplo, distintos tipos de diabetes monogénica o diabetes tipo MODY, diabetes neonatal), Defectos genéticos en la acción de insulina, Enfermedades del páncreas exocrino (por ejemplo, fibrosis quística), Endocrinopatías (por ejemplo, acromegalia, síndrome de Cushing), Inducida por fármacos o químicos (por ejemplo, pentamidina, ácido nicotínico, glucocorticoides), Infecciones, Causas poco comunes mediadas por mecanismos inmunes, Otros síndromes genéticos asociados con diabetes.La diabetes mellitus tipo 1 o DM1 corresponde a la llamada antiguamente diabetes insulino-dependiente o tipo I (DMID), diabetes de comienzo juvenil o diabetes infantojuvenil. No se observa producción de insulina debido a la destrucción de las células β de los Islotes de Langerhans del páncreas. La destrucción suele ocurrir en plazos relativamente cortos, especialmente en los más jóvenes, predisponiendo a una descompensación grave del metabolismo llamada cetoacidosis. Es más frecuente en personas jóvenes (por debajo de los 25 años) y afecta a cerca de 4,9 millones de personas en todo el mundo, con una alta prevalencia reportada en América del Norte. Representa entre un 5 a un 10 % de las DM.El origen de la DM1 es la destrucción autoinmune de las células β de páncreas, aunque en un 10 % de los casos no se han encontrado marcadores de inmunidad y se denominan idiopáticos. Esta última situación se ha visto en la casuística de DM1 de África y Asia.Dentro de las DM1 se incluye la diabetes autoinmune latente del adulto o LADA, que si bien epidemiológica y clínicamente se asemeja a la DM2, la causa es autoinmune y el tratamiento necesario es insulina. Entre un 2 a un 12 % de las diabetes en adultos corresponde a esta variante. Aproximadamente un 6 % de los pacientes con diabetes tipo 1 padece enfermedad celíaca asociada,​ si bien la mayoría de los casos de enfermedad celíaca permanecen sin reconocer ni diagnosticar, debido a que suele cursar sin síntomas digestivos, que son leves o se atribuyen erróneamente al mal control de la diabetes, a una gastroparesia o a una neuropatía diabética.La diabetes y la celíaca comparten genética común, y la inflamación o las deficiencias nutricionales causadas por la enfermedad celíaca no tratada aumentan el riesgo de desarrollar diabetes tipo 1.​ Las personas que presentan ambas enfermedades asociadas tienen peor control de los niveles de glicemia y un aumento del riesgo de complicaciones, como los daños en la retina y en el riñón, y la mortalidad.​ La dieta sin gluten, cuando se realiza estrictamente, mejora los síntomas de la diabetes y tiene un efecto protector contra el desarrollo de complicaciones a largo plazo.La diabetes mellitus tipo 2 o DM2 representa entre un 90 a un 95 % de las diabetes. Se caracteriza por una resistencia a la insulina y, al menos inicialmente, en una deficiencia relativa en la secreción de insulina, cuya cantidad, si bien es elevada en comparación con una persona normal, es insuficiente en relación con los niveles elevados de glucemia. A medida que la enfermedad avanza, el páncreas puede llegar a producir menos insulina y fallar las células beta. A diferencia de la DM1, no hay un componente autoinmune presente, si bien se mantiene un ambiente inflamatorio a nivel del tejido adiposo.Se presenta principalmente en mayores de 40 años aunque ya cada vez generaciones más jóvenes la presentan, la mayoría de los cuales presenta obesidad abdominal como parte del patrón de exceso de depósito de grasa que presentan las personas con resistencia insulína. Se acompaña frecuentemente de otras anormalidades, como hipertensión arterial, dislipidemia, disfunción del endotelio vascular y elevación del PAI-1. Este conjunto de anormalidades se ha denominado «síndrome de resistencia a la insulina» o síndrome metabólico.El diagnóstico suele hacerse transcurridos varios años desde el inicio de la enfermedad, ya que la hiperglicemia se desarrolla gradualmente y en su período inicial no produce síntomas notables. Sin embargo, durante este período los pacientes pueden desarrollar cualquiera de las complicaciones macrovasculares o microvasculares de la enfermedad. Durante este período inicial se presenta una condición denominada intolerancia a la glucosa.La diabetes mellitus gestacional o DMG es aquella que aparece en el segundo o tercer trimestre del embarazo en mujeres sin el diagnóstico previo de DM. Se presenta generalmente en mujeres mayores, hispanoamericanas, afroamericanas, asiático-americanas, indígenas pima o nativas de las islas del Pacífico.Es el nivel más alto de azúcar en la sangre que se presenta en cualquier momento del embarazo de una mujer que no tiene diabetes.Cabe destacar que la DMG es el único tipo de diabetes potencialmente reversible, puesto que el parto o cesárea pueden llevar a la resolución total de la condición orgánica. En caso de transitar un nuevo embarazo posterior, la probabilidad de padecer DMG es mayor con respecto a la población de madres que no lo han hecho.La DMG representa riesgos para la madre y el recién nacido, siendo el aumento de la glicemia entre las 24 y 28 semanas del embarazo un indicador de mayor probabilidad de complicaciones.Durante el embarazo se producen grandes cambios en el metabolismo, puesto que el feto utiliza la energía de la madre para alimentarse, oxígeno, entre otros. Esto conlleva a tener disminuida la insulina, provocando esta enfermedad.Cabe mencionar que se tiene mayor riesgo de padecer este tipo de diabetes si se tiene más de 25 años al quedar embarazada, antecedentes familiares de diabetes, hipertensión arterial, demasiado líquido amniótico, si ha tenido un aborto espontáneo o mortinato de manera inexplicable, sobrepeso antes del embarazo, o aumento excesivo de peso durante el embarazo."
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
La diabetes mellitus (DM) es un conjunto de trastornos metabólicos,
cuya característica común principal es la presencia de concentraciones
elevadas de glucosa en la sangre de manera persistente o crónica,
debido ya sea a un defecto en la producción de insulina, a una
resistencia a la acción de ella para utilizar la glucosa, a un aumento
en la producción de glucosa o a una combinación de estas causas.
También se acompaña de anormalidades en el metabolismo de los lípidos,
proteínas, sales minerales y electrolitos.La diabetes se asocia con la
aparición de complicaciones en muchos sistemas orgánicos, siendo los
más evidentes la pérdida de la visión (retinopatía diabética) que
puede llegar a la ceguera, el compromiso de los riñones (nefropatía
diabética) con deterioro funcional progresivo, requiriendo diálisis y
trasplante, el compromiso de vasos sanguíneos que pueden significar la
pérdida de extremidades inferiores (amputaciones) (véase vasculopatía
diabética), 

In [ ]:
contexto = "La artrodesis consiste en una intervención quirúrgica en la cual se fijan dos piezas óseas, anclando una articulación. Esta intervención es muy común en el raquis, fijando dos vértebras entre sí por los cuerpos vertebrales generando el anclaje por medio de una prótesis de titanio u objetos elásticos, esto último se realiza en fracturas inestables, evitándose en las estables.Este procedimiento, es también utilizado frecuentemente como tratamiento para las articulaciones afectadas por hemorragias de repetición en pacientes hemofílicos, las cuales causan un deterioro articular."
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
La artrodesis consiste en una intervención quirúrgica en la cual se
fijan dos piezas óseas, anclando una articulación. Esta intervención
es muy común en el raquis, fijando dos vértebras entre sí por los
cuerpos vertebrales generando el anclaje por medio de una prótesis de
titanio u objetos elásticos, esto último se realiza en fracturas
inestables, evitándose en las estables.Este procedimiento, es también
utilizado frecuentemente como tratamiento para las articulaciones
afectadas por hemorragias de repetición en pacientes hemofílicos, las
cuales causan un deterioro articular.

Pregunta:
-----------------
en donde es muy frecuente la artrodesis

Respuesta:
-----------------
en el raquis

Pregunta:
-----------------



In [ ]:
# PDC NM 010372 -NEURO ESTIMULACIÓN MEDULAR. ELECTRODOS PARA ESTIMULACIÓN:
contexto= "La neuroestimulación medular (NM) es una forma de tratamiento del dolor crónico que ha demostrado su efectividad en pacientes que han respondido mal a otras terapias.Se basa En que la señal transportada a lo largo de las fibras gruesas para tacto, presión y vibración puede inhibir la señal de dolor transportada por fibras más finas. La estimulación de las fibras alfa y beta mielinizadas y no nociceptivas, inhiben la transmisión del dolor por parte de las fibras alfa delta y C. Por lo tanto, la neuroestimulación medular (NM) consiste en la implantación de uno o varios electrodos epidurales en las columnas posteriores de la médula espinal al nivel de los dermatomas en los que se quiere conseguir el efecto analgésico. La estimulación de la médula espinal por pequeños impulsos eléctricos, bloquean la transmisión de mensajes de dolor al cerebro. Los pacientes pueden sentir una leve sensación de hormigueo en el lugar del dolor. Los neuroestimuladores implantables tienen un tamaño similar a un marcapasos, estos dispositivos envían la estimulación eléctrica a través de un cable de electrodos implantados cerca de la médula espinal o los nervios periféricos afectados, incluyen un circuito integrado (un ordenador en miniatura), un transmisor receptor de ondas de radio, una batería y un bloque conector. Se puede ajustar la intensidad y la localización de la estimulación usando un programador o dispositivo de mano. Puede regular distintos niveles de estimulación en los diferentes momentos del día o para diversas actividades como caminar, dormir o estar sentado personalizando la estimulación. Las indicaciones actuales para la neuroestimulación medular son las terapias en el dolor crónico común incluyendo el dolor axial por síndrome de cirugía fallida de espalda o por dolor radicular, (por lo menos 6 meses de evolución, en aquellos operados de aracnoiditis adhesivas o en el Síndrome de la Cola de Caballo, Síndrome post-laminectomía), dolor neuropático incluyendo la neuropatía diabética, neuropatía por VIH, síndrome de dolor regional complejo (SDRC), neuralgia postherpética, neuralgia intercostal, dolor de miembro fantasma, neuropatía compresiva, dolor pélvico y lesión medular, otra indicación es el dolor vascular que engloba la angina crónica refractaria, insuficiencia cardíaca, dolor isquémico por vasculopatía periférica y el síndrome de Raynaud. Las consideraciones del Dolor Neuropático Crónico son que deberá demostrarse durante un tiempo establecido y debe constar el antecedente de ser refractario a por lo menos cuatro fármacos indicados para el dolor neuropático, de no existir contraindicaciones, durante un tiempo mayor o igual a tres 3 meses, no habiendo disminuido el dolor más del 30% manteniéndose con una intensidad mayor de 5 en la escala de 0 a 10 y generando una mala calidad de vida. En el Síndrome doloroso regional complejo (SDRC) tipo I o tipo II se debe discriminar el componente simpático por bloqueo diagnóstico pronóstico previo al implante definitivo ya que tiene mejor respuesta en aquellos pacientes en los que el SDRC es medido de forma independiente del sistema simpático, dado que en SDRC la evidencia científica es de fuerte nivel para alivio a corto término (menos de 1 año) y moderado nivel para alivio de largo término (1 año o más). La mejoría que proporciona la neuroestimulación medular puede ser muy variable en función de la terapia y el origen del dolor. El dolor crónico lumbar es una de las principales indicaciones de implante de dispositivos de neuroestimulación medular. Las reacciones adversas de la neuroestimulación medular son la falta de estimulación con pérdida de alivio del dolor (debido al movimiento del electrodo u otras causas), la estimulación intermitente, la estimulación en una zona inadecuada si se mueven los electrodos, la estimulación molesta, dolor en el lugar del estimulador, problemas del programador o de transmisión de datos, riesgo de infección, hematoma o seroma en el área donde se implanta en neuroestimulador y en casos raros, puede ocurrir una lesión de la médula espinal debida a una colocación quirúrgica incorrecta. Las complicaciones del dispositivo pueden requerir cirugía adicional para recolocar, reparar o sustituir piezas del sistema. Las contraindicaciones de la neuroestimulación medular son dolores de naturaleza mecánica y de línea media cuyo ejemplo más claro es el dolor raquídeo de naturaleza espondiloartrósica, dolor por isquemia grado IV. La neuroestimulación medular no debe emplearse sin un estudio psicológico específico, se debe realizar una entrevista psicológica estructurada y un estudio de personalidad con el test multifásico de personalidad de Minnesota. Consideramos factores peligrosos para la neuroestimulación medular la presencia de trastornos psiquiátricos activos, las toxicomanías y la existencia de litigios laborales sin resolver. El sistema de reintegro nacional recupero SUR reconoce la neuroestimulación medular teniendo en cuenta los siguientes ítems evaluación multidisciplinaria previa por equipo conformado por Neurólogo, Psiquiatra, Neurocirujano y Psicólogo, demostrar en todos los casos mediante Historia Clínica que los pacientes padecen dolor crónico y se les ha efectuado tratamiento analgésico según escalera analgésica de la OMS a dosis máxima u otras técnicas sin respuesta satisfactoria, no ser considerados de alto riesgo quirúrgico en su evaluación anestésica, haber tenido alivio efectivo durante la estimulación de prueba por vía percutánea con una reducción del dolor del 50% o más, al menos durante 7 días, tener una evaluación psiquiátrica psicológica que no contraindique el procedimiento, tener una valoración previa del dolor por escala Visual analógica o cuestionario de Oswestry para documentar el nivel de intensidad o el grado de discapacidad por dolor previo. La documentación médica específica a presentar para el reintegro de la neuroestimulación medular son Indicación médica fundamentada con solicitud del tipo de neuroestimulador y detalles técnicos, firmado y sellado por el Médico Especialista y Auditor Médico del Agente del Seguro de Salud, en la Historia Clínica es indispensable que conste la valoración post-operatoria del nivel del dolor por escala visual analógica o cuestionario de Oswestry con firma del profesional interviniente y del Auditor Médico del Agente del Seguro de Salud, consentimiento informado del paciente protocolo del procedimiento y comprobante de implante del dispositivo. Las ventajas de la neuroestimulación medular son que permite realizar una prueba temporal de 3 a 7 días, para determinar como responde el paciente a la neuroestimulación antes de comprometerse con un sistema implantado. Las desventajas de la neuroestimulación medular son las complicaciones del procedimiento, y el alto costo. Las política de cobertura de la neuroestimulación medular son es una practica de exclusión medica del programa médico obligatorio (PMO), es una prestación con recupero SUR, el origen de la prestación es internación, requiere auditoría médica, no debe autorizarse junto a la prestación 010271 Neuroestimulación central. La neuroestimulación medular requiere como prótesis o material 715352 NEUROESTIMULADOR ESPINAL o 715353 - NEUROESTIMULADOR IMPLANTABLE. La normativa de la neuroestimulación medular es El dolor axial por síndrome de cirugía fallida de espalda o por dolor radicular. Síndrome post-laminectomía. Dolor neuropático incluyendo la neuropatía diabética, neuropatía por VIH, síndrome de dolor regional complejo (SDRC), neuralgia postherpética, neuralgia intercostal, dolor de miembro fantasma, neuropatía compresiva, dolor pélvico y lesión medular. Dolor vascular que engloba la angina crónica refractaria, insuficiencia cardíaca, dolor isquémico por vasculopatía periférica y el síndrome de Raynaud."
pregunta_respuesta(model, contexto, nlp)

Contexto:
-----------------
La neuroestimulación medular (NM) es una forma de tratamiento del
dolor crónico que ha demostrado su efectividad en pacientes que han
respondido mal a otras terapias.Se basa En que la señal transportada a
lo largo de las fibras gruesas para tacto, presión y vibración puede
inhibir la señal de dolor transportada por fibras más finas. La
estimulación de las fibras alfa y beta mielinizadas y no nociceptivas,
inhiben la transmisión del dolor por parte de las fibras alfa delta y
C. Por lo tanto, la neuroestimulación medular (NM) consiste en la
implantación de uno o varios electrodos epidurales en las columnas
posteriores de la médula espinal al nivel de los dermatomas en los que
se quiere conseguir el efecto analgésico. La estimulación de la médula
espinal por pequeños impulsos eléctricos, bloquean la transmisión de
mensajes de dolor al cerebro. Los pacientes pueden sentir una leve
sensación de hormigueo en el lugar del dolor. Los neuroestimuladores
implantables 